# I. Configuration du projet et initialisation de l'environnement

### 1.1. Description technique
Ce module constitue le socle structurel du projet. Il assure la centralisation des paramètres de flux de données via un dictionnaire de configuration (`config`) et l'automatisation de l'arborescence de travail. 

Le script orchestre plusieurs dimensions critiques :
* **Gestion des Répertoires** : Utilisation de la bibliothèque `pathlib` pour garantir l'agnosticisme du système d'exploitation (Windows/Linux) dans le traitement des chemins.
* **Référentiel d'Identité** : Définition d'une clé composite (`CHAMPS_ID`) incluant le champ `DALLE`, permettant une traçabilité géographique fine et un partitionnement des calculs.
* **Paramétrage Multi-sources** : Centralisation des millésimes (2017-2024) et des chemins d'accès pour les cinq sources de données majeures (OSO, RPG standard et complet, COSIA, BD FORÊT et BD TOPO).
* **Déploiement Automatisé** : Création dynamique des sous-répertoires de sortie pour assurer l'organisation des données intermédiaires (CSV).

### 1.2. Justification méthodologique
L'adoption de cette architecture centralisée répond à trois impératifs:

1. **Reproductibilité** : La séparation stricte entre les répertoires de données sources (`ENTRER`) et les sorties (`SORTIE`) permet de rejouer l'intégralité du pipeline sur d'autres territoires sans modifier la logique algorithmique.
2. **Optimisation des Performances** : L'intégration du champ `DALLE` dès l'initialisation prépare le système à des traitements par tuiles. Cette approche est indispensable pour prévenir les dépassements de mémoire vive lors des opérations d'intersection géométrique sur des jeux de données massifs (Big Data spatial).
3. **Harmonisation Temporelle** : En définissant les listes d'années par source dans une configuration unique, on garantit la cohérence chronologique lors des phases ultérieures de croisement et de calcul de stabilité de l'occupation du sol.

In [1]:
import os
import geopandas as gpd
from pathlib import Path

# =============================================================================
# 1. DÉFINITION DES RÉPERTOIRES RACINES
# =============================================================================
DIR_ENTREE_SIG   = Path(r'C:/Users/liege/Desktop/GEOLAB/ENTRER')
DIR_ENTREE_NOMEN = Path(r'C:/Users/liege/Desktop/GEOLAB/NOMENCLATURE')
DIR_SORTIE       = Path(r'C:/Users/liege/Desktop/GEOLAB/SORTIE')

# =============================================================================
# 2. CONFIGURATION GÉNÉRALE DES FLUX DE DONNÉES
# =============================================================================
config = {
    # Ajout du champ "DALLE" pour assurer la traçabilité géographique par secteur
    'CHAMPS_ID': ["CODE_DEP", "NOM_COM", "CODE_COM", "SECTION", "NUMERO", "DALLE"],
    
    # Référentiels spatiaux de base
    'ZONE_PATH'     : DIR_ENTREE_SIG / "ZONE.gpkg",
    'CADASTRE_PATH' : DIR_ENTREE_SIG / "CADASTRE.gpkg",
    
    # Configuration OSO (Occupation du Sol - CESBIO)
    'OSO_GPKG'       : DIR_ENTREE_SIG / "OSO_2018_2023_ALL.gpkg",
    'NOMEN_OSO_CSV'  : DIR_ENTREE_NOMEN / "Nomenclature_OSO.csv",
    'ANNEES_OSO'     : ["2018", "2019", "2020", "2021", "2022", "2023"],
    'DOSSIER_CSV_OSO': DIR_SORTIE / "CSV_OSO",
    
    # Configuration RPG (Fusion des sources Standard et Complet)
    'RPG_GPKG'        : DIR_ENTREE_SIG / "RPG_2018_2024_ALL.gpkg",
    'RPG_COMP_GPKG'   : DIR_ENTREE_SIG / "RPG_COMPLETE_2018_2022_ALL.gpkg",
    'ANNEES_RPG'      : ["2018", "2019", "2020", "2021", "2022", "2023", "2024"],
    'ANNEES_RPG_COMP' : ["2018", "2019", "2020", "2021", "2022"],
    'DIR_CSV_RPG_FUS' : DIR_SORTIE / "CSV_RPG_FUSION",

    # Configuration COSIA
    'COSIA_GPKG'       : DIR_ENTREE_SIG / "COSIA_2018_2024_ALL.gpkg",
    'NOMEN_COSIA_CSV'  : DIR_ENTREE_NOMEN / "Nomenclature_COSIA.csv",
    'ANNEES_COSIA'     : ["2018", "2021", "2024"],
    'DOSSIER_CSV_COSIA': DIR_SORTIE / "CSV_COSIA",

    # Configuration BD FORÊT (IGN)
    'FORET_PATH'       : DIR_ENTREE_SIG / "FORMATION_VEGETALE_BD_FORET_2017.gpkg",
    'DOSSIER_CSV_FORET': DIR_SORTIE / "CSV_FORET",
    
    # Configuration BD TOPO (IGN)
    'BDTOPO_PATH'       : DIR_ENTREE_SIG / "BDTOPO_2025.gpkg",
    'DOSSIER_CSV_BDTOPO': DIR_SORTIE / "CSV_BDTOPO",
    'COUCHES_BDTOPO'    : [
        "BATIMENT", "CIMETIERE", "CONSTRUCTION_SURFACIQUE", 
        "SURFACE_HYDROGRAPHIQUE", "RESERVOIR", "TERRAIN_DE_SPORT", 
        "ZONE_D_ESTRAN"
    ]
}

# =============================================================================
# 3. INITIALISATION DE L'ARBORESCENCE DE SORTIE
# =============================================================================
dossiers_requis = [
    DIR_SORTIE,
    config['DOSSIER_CSV_OSO'],
    config['DIR_CSV_RPG_FUS'],
    config['DOSSIER_CSV_COSIA'],
    config['DOSSIER_CSV_FORET'],
    config['DOSSIER_CSV_BDTOPO']
]

for dossier in dossiers_requis:
    dossier.mkdir(parents=True, exist_ok=True)

print("Environnement initialisé : Prêt pour le traitement multi-sources avec gestion des dalles.")

Environnement initialisé : Prêt pour le traitement multi-sources avec gestion des dalles.


# II. Extraction du référentiel parcellaire et indexation spatiale

### 2.1. Description technique
Cette étape constitue le socle géographique de l'étude par la création d'un référentiel foncier précis. Le processus s'articule autour de deux opérations majeures :
* **Filtrage spatial à la source** : Chargement ciblé du cadastre national à l'aide d'un masque (`mask`) basé sur l'emprise d'étude. L'utilisation du moteur `pyogrio` garantit une lecture performante des géométries complexes.
* **Indexation par jointure spatiale (`SJoin`)** : Enrichissement de la donnée parcellaire par l'injection de l'attribut `DALLE`. Cette opération de voisinage (`predicate='intersects'`) permet de localiser chaque parcelle dans le système de partitionnement du projet.

### 2.2. Justification de l'approche
Le recours à cette méthode d'extraction et d'indexation répond à des impératifs d'optimisation et de traçabilité :

1. **Intégrité des données** : L'extraction par masque garantit que seules les parcelles intersectant la zone d'intérêt sont conservées, éliminant ainsi les bruits de fond et les données superflues hors périmètre d'étude.
2. **Gestion de la volumétrie (Tiling)** : L'attribution d'un numéro de `DALLE` à chaque parcelle est stratégique. Elle permettrait de fragmenter les futurs calculs statistiques par secteur, évitant ainsi la saturation de la mémoire vive sur des analyses multi-annuelles.
3. **Traçabilité géographique** : Cette indexation offre une double lecture du territoire : administrative (via les codes parcelles) et opérationnelle (via le découpage par dalles), indispensable pour le contrôle qualité des traitements SIG.

In [2]:
# =============================================================================
# 2. EXTRACTION DU RÉFÉRENTIEL PARCELLAIRE ET JOINTURE DALLE
# =============================================================================
# Chargement de l'emprise d'étude et découpage du cadastre national.

print("Chargement de l'emprise et extraction du cadastre...")

gdf_zone = gpd.read_file(config['ZONE_PATH'])

# Chargement du cadastre avec le masque de la zone
gdf_parcelles = gpd.read_file(
    config['CADASTRE_PATH'], 
    mask=gdf_zone,
    engine='pyogrio'
)

# --- AJOUT DE LA JOINTURE POUR LE NUMÉRO DE DALLE ---
# On récupère l'attribut 'DALLE' de la zone pour l'injecter dans les parcelles
gdf_parcelles = gpd.sjoin(gdf_parcelles, gdf_zone[['DALLE', 'geometry']], how='left', predicate='intersects')

# Nettoyage de l'index de jointure (colonne technique inutile)
if 'index_right' in gdf_parcelles.columns:
    gdf_parcelles = gdf_parcelles.drop(columns='index_right')
# ----------------------------------------------------

print(f"Référentiel prêt : {len(gdf_parcelles)} parcelles chargées avec leur numéro de DALLE.")

Chargement de l'emprise et extraction du cadastre...
Référentiel prêt : 6792 parcelles chargées avec leur numéro de DALLE.


# III. Analyse multi-annuelle et synthèse de l'occupation du sol (pour OSO)

### 3.1. Module 1 : Analyse statistique annuelle par unité foncière
Ce premier module opère une synthèse annuelle de l'occupation du sol. Le processus est structuré autour de l'intersection géométrique entre le cadastre et les données OSO.
* **Intersection spatiale (`Overlay`)** : Découpage des couches vectorielles annuelles par le référentiel parcellaire. Chaque fragment résultant permet de calculer précisément la surface occupée par chaque classe au sein de la parcelle.
* **Calcul de dominance et fiabilité** : Pour chaque millésime, le script identifie la classe majoritaire et calcule sa **probabilité pondérée** :
  $$PROBA\_DOMINANTE = \frac{\sum (Surface \cap \times Confidence)}{\sum Surface \cap}$$
  Cette méthode garantit que la confiance affichée est proportionnelle à la surface réelle occupée par la classe, minimisant ainsi l'impact des artefacts de classification en bordure de parcelle.



### 3.2. Module 2 : Synthèse temporelle et arbitrage statistique
L'objectif de ce second module est de dégager une tendance de long terme (2018-2023) pour chaque parcelle en s'affranchissant des erreurs ponctuelles de détection satellite ou IA.
* **Consolidation chronologique** : Fusion des résultats annuels via l'identifiant unique `PL` pour créer un historique complet de l'occupation du sol par unité foncière.
* **Logique de décision par majorité fréquentielle** : Le système analyse l'occurrence de chaque classe sur 6 ans. La classe la plus fréquente est élue comme `DOM_OSO_FINALE`.
* **Arbitrage par récence** : En cas d'égalité de fréquence (ex: 3 ans en "Forêt" et 3 ans en "Agriculture"), le script applique une règle de **chronologie inverse**. Il interroge les données en partant de l'année la plus récente (2023) pour trancher en faveur de l'état le plus actuel du terrain.
* **Indicateur de stabilité et fiabilité moyenne** :
    * **Score de stabilité** : Pourcentage de présence de la classe élue sur la période étudiée.
    * **Proba moyenne** : Moyenne des indices de confiance calculés lors du Module 1, uniquement pour les années où la classe élue était présente.



### 3.3. Justification Méthodologique
Cette approche en deux étapes permet de produire un diagnostic territorial **résilient**. En combinant la précision spatiale annuelle (Module 1) et la stabilité temporelle (Module 2), le modèle final est capable de distinguer les changements réels d'occupation du sol des simples "bruits" de capteurs ou de variations saisonnières.

In [15]:
# =============================================================================
# MODULE : ANALYSE STATISTIQUE DE L'OCCUPATION DU SOL (OSO)
# =============================================================================
# Ce module realise l'intersection spatiale entre le referentiel parcellaire 
# et les couches annuelles OSO. Il genere un identifiant unique 'PL' par 
# concatenation textuelle directe des composantes cadastrales.

import pandas as pd
import geopandas as gpd

# --- 1. PREPARATION DES REFERENTIELS ET NOMENCLATURES ---
classes_list = ["ARTIFICIALISE", "AGRICULTURE", "NATURE", "BOIS", "EAU", "AUTRE"]
noms_classes_map = {"1":"ARTIFICIALISE", "2":"AGRICULTURE", "4":"NATURE", "3":"BOIS", "5":"EAU", "0":"AUTRE"}

df_nomen = pd.read_csv(config['NOMEN_OSO_CSV'], sep=';', encoding='utf-8-sig')
mapping_oso = {str(row['CODE_SOURCE']).strip(): noms_classes_map.get(str(row['CLASSE_6']).strip(), "AUTRE") 
               for _, row in df_nomen.iterrows()}

# --- 2. TRAITEMENT ANALYTIQUE ANNUEL ---
for annee in config['ANNEES_OSO']:
    print(f"INFO : Analyse spatiale OSO - Millesime {annee}")
    
    lyr_oso = None
    # Recherche itérative de la couche selon les variantes de nommage possibles
    for layer_name in [f"OSO_VECTEUR_{annee}", f"OSO_{annee}", f"VECTEUR_{annee}"]:
        try:
            lyr_oso = gpd.read_file(config['OSO_GPKG'], layer=layer_name, mask=gdf_parcelles, engine='pyogrio')
            if lyr_oso.crs != gdf_parcelles.crs:
                lyr_oso = lyr_oso.to_crs(gdf_parcelles.crs)
            break
        except:
            continue

    if lyr_oso is None:
        print(f"ERREUR : Couche OSO {annee} introuvable dans le GeoPackage.")
        continue

    # Operation d'intersection geometrique
    inter = gpd.overlay(lyr_oso, gdf_parcelles, how='intersection')
    if inter.empty:
        continue
    
    # Calcul des surfaces et identification des attributs sources
    inter['S_INT'] = inter.geometry.area
    f_classe = "Classe" if "Classe" in inter.columns else "CLASSE"
    f_conf = next((f for f in inter.columns if f.lower() == "confidence"), None)
    inter['CLASSE_NOM'] = inter[f_classe].astype(str).str.strip().map(mapping_oso).fillna("AUTRE")

    # Agregation statistique par unite fonciere
    stats = inter.groupby(config['CHAMPS_ID'] + ['CLASSE_NOM']).agg({
        'S_INT': 'sum',
        f_conf: lambda x: (x * inter.loc[x.index, 'S_INT']).sum() if f_conf else 0
    }).reset_index()

    # Pivotage des donnees pour obtenir une structure par ligne de parcelle
    df_res = stats.pivot_table(index=config['CHAMPS_ID'], columns='CLASSE_NOM', values='S_INT', aggfunc='sum').fillna(0)
    
    for cl in classes_list:
        if cl not in df_res.columns: df_res[cl] = 0.0

    # Calcul des indicateurs de synthese avec precision a 3 decimales
    df_res['SURF_TOTALE_M2'] = df_res[classes_list].sum(axis=1).round(3)
    for cl in classes_list:
        df_res[f"{cl}_%"] = ((df_res[cl] / df_res['SURF_TOTALE_M2']) * 100).round(3)

    df_res['DOMINANTE'] = df_res[classes_list].idxmax(axis=1)

    # Calcul de la fiabilite ponderee sur la classe dominante
    conf_pivot = stats.pivot_table(index=config['CHAMPS_ID'], columns='CLASSE_NOM', values=f_conf, aggfunc='sum').fillna(0)
    def get_proba(row):
        dom = row['DOMINANTE']
        s_dom = row[dom]
        cumul = conf_pivot.loc[row.name, dom] if row.name in conf_pivot.index else 0
        return round(cumul / s_dom, 1) if s_dom > 0 else 0

    df_res['PROBA_DOMINANTE'] = df_res.apply(get_proba, axis=1)

    # --- 3. GENERATION DU CHAMP 'PL' (CONCATENATION SIMPLE) ---
    df_res = df_res.reset_index()
    
    # Concaténation des 4 champs racines sans préfixe intermédiaire
    # Le passage en .astype(str) garantit le maintien du format texte
    df_res['PL'] = (
        df_res['CODE_DEP'].astype(str).str.strip() + 
        df_res['CODE_COM'].astype(str).str.strip() + 
        df_res['SECTION'].astype(str).str.strip() + 
        df_res['NUMERO'].astype(str).str.strip()
    )

    # --- 4. EXPORTATION DES RESULTATS ---
    output_csv = config['DOSSIER_CSV_OSO'] / f"STAT_PARCELLES_OSO_{annee}.csv"
    cols_finales = ['PL'] + config['CHAMPS_ID'] + [f"{cl}_%" for cl in classes_list] + \
                   ["DOMINANTE", "PROBA_DOMINANTE", "SURF_TOTALE_M2"]
    
    df_res[cols_finales].to_csv(output_csv, sep=';', index=False, encoding='utf-8')
    print(f"SUCCES : Export millesime {annee} genere.")

print("TRAITEMENT OSO TERMINE.")

INFO : Analyse spatiale OSO - Millesime 2018
SUCCES : Export millesime 2018 genere.
INFO : Analyse spatiale OSO - Millesime 2019
SUCCES : Export millesime 2019 genere.
INFO : Analyse spatiale OSO - Millesime 2020
SUCCES : Export millesime 2020 genere.
INFO : Analyse spatiale OSO - Millesime 2021
SUCCES : Export millesime 2021 genere.
INFO : Analyse spatiale OSO - Millesime 2022
SUCCES : Export millesime 2022 genere.
INFO : Analyse spatiale OSO - Millesime 2023
SUCCES : Export millesime 2023 genere.
TRAITEMENT OSO TERMINE.


In [3]:
# =============================================================================
# MODULE : SYNTHESE TEMPORELLE ET ARBITRAGE OSO
# =============================================================================
# Analyse de la stabilite temporelle des classes dominantes par parcelle.
# Application de la regle de majorite statistique avec arbitrage par recence.

import os
import csv
from collections import defaultdict

# --- 1. CONFIGURATION ---
# Utilisation des cles exactes de votre bloc initial
fichier_synthese = DIR_SORTIE / "SYNTHESE_OSO_FREQUENCE.csv"
CHAMPS_ID = config['CHAMPS_ID']
ANNEES_LISTE = sorted([int(a) for a in config['ANNEES_OSO']])

# Structure de stockage des donnees chronologiques
base_globale = defaultdict(dict)

# --- 2. CONSOLIDATION DES DONNEES ANNUELLES ---
print("INFO : Phase de lecture des fichiers annuels OSO...")
for annee in config['ANNEES_OSO']:
    # Utilisation du dossier specifie dans votre config
    chemin = config['DOSSIER_CSV_OSO'] / f"STAT_PARCELLES_OSO_{annee}.csv"
    
    if not chemin.exists():
        continue
        
    with open(chemin, mode='r', encoding='utf-8') as f:
        reader = csv.DictReader(f, delimiter=';')
        for row in reader:
            # Recuperation securisee du champ PL ou creation par concatenation
            if 'PL' in row:
                id_pl = str(row['PL']).strip()
            else:
                # Securite si le champ PL n'a pas ete genere dans le module precedent
                id_pl = "".join([str(row[c]).strip() for c in CHAMPS_ID])
                
            base_globale[id_pl][int(annee)] = {
                'DOM': row['DOMINANTE'],
                'PROB': float(row['PROBA_DOMINANTE']) if row['PROBA_DOMINANTE'] else 0.0,
                'ID_ATTRS': [row[c] for c in CHAMPS_ID]
            }

# --- 3. CALCUL DE LA DECISION FINALE ---
print("INFO : Calcul des frequences et arbitrage par chronologie inverse...")
with open(fichier_synthese, mode='w', newline='', encoding='utf-8') as f_out:
    w = csv.writer(f_out, delimiter=';')
    
    # Construction de l'entete de sortie
    entete = ["PL"] + CHAMPS_ID
    for a in ANNEES_LISTE:
        entete += [f"DOM_{a}", f"PROB_{a}"]
    w.writerow(entete + ["DOM_OSO_FINALE", "SCORE_STABILITE", "PROBA_MOY_ELU", "METHODE"])

    for id_pl, data in base_globale.items():
        historique_classes = [data[a]['DOM'] for a in ANNEES_LISTE if a in data]
        if not historique_classes:
            continue

        # Comptage des occurrences par classe
        counts = defaultdict(int)
        for c in historique_classes:
            if c and c != "N/A":
                counts[c] += 1
        
        if not counts:
            continue

        # Determination de la frequence maximale
        max_freq = max(counts.values())
        gagnants = [cl for cl, nb in counts.items() if nb == max_freq]

        # Logique d'arbitrage (Majorite puis Recence)
        if len(gagnants) == 1:
            elu = gagnants[0]
            methode = "MAJORITE_FREQUENCE"
        else:
            elu = None
            for a_check in reversed(ANNEES_LISTE):
                classe_annee = data.get(a_check, {}).get('DOM')
                if classe_annee in gagnants:
                    elu = classe_annee
                    break
            methode = "ARBITRAGE_DATE_RECENTE"

        # Calcul des indicateurs de fiabilite
        score_stabilite = round((counts[elu] / len(ANNEES_LISTE)) * 100, 1)
        probas_elu = [data[a]['PROB'] for a in data if data[a].get('DOM') == elu]
        proba_moyenne = round(sum(probas_elu) / len(probas_elu), 1) if probas_elu else 0.0

        # Preparation de l'export
        premiere_date = next(iter(data))
        identifiants = data[premiere_date]['ID_ATTRS']
        
        ligne = [id_pl] + identifiants
        for a in ANNEES_LISTE:
            ligne += [data.get(a, {}).get('DOM', "N/A"), data.get(a, {}).get('PROB', 0.0)]
        
        ligne += [elu, score_stabilite, proba_moyenne, methode]
        w.writerow(ligne)

print(f"INFO : Synthese terminee. Resultats : {fichier_synthese.name}")

INFO : Phase de lecture des fichiers annuels OSO...
INFO : Calcul des frequences et arbitrage par chronologie inverse...
INFO : Synthese terminee. Resultats : SYNTHESE_OSO_FREQUENCE.csv


# IV. Intégration du référentiel BD FORÊT V2 2017 (source IGN)

### 4.1. Description technique
Ce module exploite la base de données forestière de l'IGN (BD FORÊT® v2) pour caractériser l'occupation forestière et semi-naturelle.
* **Typologie des formations végétales (TFV)** : Le script utilise une fonction de classification (`classifier_tfv`) pour harmoniser la nomenclature IGN. Dans cette version, les formations **"herbacées"** ainsi que les **"landes"** sont rattachées à la classe **NATURE**, tandis que les formations arborées sont classées en **BOIS**.
* **Calcul du Reliquat ("AUTRE")** : Le script identifie par soustraction les espaces de la parcelle non couverts par la BD FORÊT, permettant de maintenir une cohérence surfacique totale par rapport au référentiel cadastral.

### 4.2. Logique de fiabilité et domaine de compétence
La gestion de la probabilité de confiance dans ce module repose sur une approche binaire stricte :
* **Confiance Maximale (100)** : Une valeur de 100 est attribuée aux classes **BOIS** et **NATURE**. En l'absence de contre-indication sur ce référentiel institutionnel, on considère la donnée IGN comme une "vérité terrain" absolue pour ces thématiques.
* **Neutralité de Compétence (0)** : La valeur de probabilité est fixée à **0** pour la classe **AUTRE**. Ce choix méthodologique souligne que la BD FORÊT n'est pas compétente pour qualifier les espaces non forestiers (bâti, agriculture, etc.). Cela permet, lors de la fusion finale, de ne pas donner de poids à une information de "non-présence" issue d'une source spécialisée.

### 4.3. Justification Méthodologique
1. **Précision de l'overlay** : L'utilisation d'une intersection géométrique stricte (`gpd.overlay`) assure un calcul de surface au mètre carré près.
2. **Référentiel de référence** : La BD FORÊT sert de base stable pour confirmer la présence historique de végétation pérenne.

In [3]:
# =============================================================================
# MODULE : ANALYSE STATISTIQUE BD FORÊT (IGN)
# =============================================================================
import pandas as pd
import geopandas as gpd

print("INFO : Analyse spatiale BD FORET - Millesime 2017")

# --- 1. CHARGEMENT ET SECURISATION DES TYPES ---
for col in config['CHAMPS_ID']:
    if col in gdf_parcelles.columns:
        gdf_parcelles[col] = gdf_parcelles[col].astype(str).str.strip()

try:
    lyr_foret = gpd.read_file(
        config['FORET_PATH'], 
        layer="FORMATION_VEGETALE_BD_FORET_2017", 
        mask=gdf_parcelles, 
        engine='pyogrio'
    )
    if lyr_foret.crs != gdf_parcelles.crs:
        lyr_foret = lyr_foret.to_crs(gdf_parcelles.crs)
except Exception as e:
    print(f"ERREUR : Impossible de charger la couche Foret : {e}")
    lyr_foret = None

if lyr_foret is not None:
    # --- 2. TRAITEMENT GEOMETRIQUE ---
    inter = gpd.overlay(lyr_foret, gdf_parcelles, how='intersection')
    
    if not inter.empty:
        inter['S_INT'] = inter.geometry.area
        
        def classifier_tfv(tfv):
            tfv_str = str(tfv).lower()
            if "lande" in tfv_str: return "NATURE"
            elif "herbacée" in tfv_str: return "NATURE"
            else: return "BOIS"

        inter['CLASSE_NOM'] = inter['TFV'].apply(classifier_tfv)

        # Agregation par parcelle
        stats = inter.groupby(config['CHAMPS_ID'] + ['CLASSE_NOM'])['S_INT'].sum().reset_index()
        df_res = stats.pivot_table(index=config['CHAMPS_ID'], columns='CLASSE_NOM', values='S_INT').fillna(0)

        # --- 3. CALCUL DES INDICATEURS ---
        classes_foret = ["BOIS", "NATURE"]
        for cl in classes_foret:
            if cl not in df_res.columns: df_res[cl] = 0.0

        df_res = df_res.reset_index()
        surf_tot = gdf_parcelles[config['CHAMPS_ID'] + ['geometry']].copy()
        
        # Fusion pour recuperer la géométrie de reference
        df_res = pd.merge(df_res, surf_tot, on=config['CHAMPS_ID'], how='right').fillna(0)
        
        # SECURITE : On force la conversion en GeoDataFrame pour pouvoir utiliser .area
        df_res = gpd.GeoDataFrame(df_res, geometry='geometry', crs=gdf_parcelles.crs)
        
        # Calcul de la surface totale arrondie
        df_res['SURF_TOTALE_M2'] = df_res.geometry.area.round(1)

        # Calcul de la classe AUTRE (Reste de la parcelle)
        df_res['AUTRE'] = (df_res['SURF_TOTALE_M2'] - df_res[classes_foret].sum(axis=1)).clip(lower=0)
        
        # Calcul des pourcentages (arrondis a 2 decimales)
        classes_finales = ["BOIS", "NATURE", "AUTRE"]
        for cl in classes_finales:
            df_res[f"{cl}_%"] = ((df_res[cl] / df_res['SURF_TOTALE_M2']) * 100).round(2)

        # Determination de la dominante
        df_res['DOMINANTE'] = df_res[[f"{cl}_%" for cl in classes_finales]].idxmax(axis=1).str.replace('_%', '')

        # --- 4. COLONNE PROBA (PROD) ---
        df_res['PROBA_DOMINANTE'] = df_res['DOMINANTE'].apply(lambda x: 100 if x != "AUTRE" else 0)

        # --- 5. GENERATION DU CHAMP 'PL' (TEXTE PUR) ---
        df_res['PL'] = (
            df_res['CODE_DEP'].astype(str) + 
            df_res['CODE_COM'].astype(str) + 
            df_res['SECTION'].astype(str) + 
            df_res['NUMERO'].astype(str)
        )

        # --- 6. EXPORTATION ---
        output_csv = config['DOSSIER_CSV_FORET'] / "STAT_PARCELLES_FORET_2017.csv"
        cols_finales = ['PL'] + config['CHAMPS_ID'] + [f"{cl}_%" for cl in classes_finales] + \
                       ["DOMINANTE", "PROBA_DOMINANTE", "SURF_TOTALE_M2"]
        
        df_res[cols_finales].to_csv(output_csv, sep=';', index=False, encoding='utf-8')
        print(f"SUCCES : Fichier BD FORET genere -> {output_csv.name}")

print("TRAITEMENT BD FORET TERMINE.")

INFO : Analyse spatiale BD FORET - Millesime 2017
SUCCES : Fichier BD FORET genere -> STAT_PARCELLES_FORET_2017.csv
TRAITEMENT BD FORET TERMINE.


# V. Caractérisation par le référentiel BD TOPO (source IGN)

### 5.1. Description technique : Approche multi-couches
Ce module exploite la précision métrique de la BD TOPO® pour affiner le diagnostic territorial. Contrairement aux modules précédents, il traite un ensemble hétérogène de thématiques :
* **Objets structuraux** : Intersection systématique avec les couches de bâti, d'équipements (cimetières, sport) et d'hydrographie. Chaque couche est redirigée vers une classe cible (ex: Bâtiment → **ARTIFICIALISE**).
* **Cohérence spatiale** : Utilisation d'un `overlay` par parcelle pour calculer le taux d'emprise précis de chaque infrastructure ou milieu naturel.

### 5.2. Logique de fiabilité et domaine de compétence
La gestion de la confiance suit la même rigueur méthodologique que le module forestier :
* **Confiance institutionnelle (100)** : S'agissant du référentiel topographique de référence en France, une probabilité de 100 est assignée à toutes les classes identifiées par l'IGN.
* **Neutralité du reliquat (0)** : La classe **AUTRE** (surface non couverte par une couche BD TOPO) reçoit une probabilité de **0**. Cela signifie que la BD TOPO ne se prononce pas sur l'occupation de ces espaces, laissant le champ libre aux autres sources (ex : OSO/RPG) lors de la fusion finale.

### 5.3. Justification méthodologique
1. **Précision du bâti** : La BD TOPO est la seule source capable de garantir la détection des petites structures artificialisées (réservoirs, constructions secondaires) souvent invisibles par satellite.
2. **Identification des zones humides et sportives** : Elle apporte des précisions sémantiques cruciales sur les plans d'eau et les terrains de sport, souvent confondus avec d'autres classes dans les classifications automatiques.

In [3]:
# =============================================================================
# MODULE : ANALYSE STATISTIQUE BD TOPO (IGN)
# =============================================================================
import pandas as pd
import geopandas as gpd
import numpy as np
from pathlib import Path

print(">>> DÉMARRAGE DE L'ANALYSE BD TOPO (SANS VÉGÉTATION)")

# 1. PARAMÉTRAGE ET INITIALISATION
C_ID = config['CHAMPS_ID']  
TOPO_PATH = config['BDTOPO_PATH']
OUT_DIR = config['DOSSIER_CSV_BDTOPO']

if 'gdf_parcelles' not in locals():
    raise EnvironmentError("Référentiel 'gdf_parcelles' absent. Exécuter le Module 0.")

# Définition des classes cibles
classes_list = ["ARTIFICIALISE", "AGRICULTURE", "NATURE", "BOIS", "EAU", "AUTRE"]

# Initialisation du GeoDataFrame de calcul
df_travail = gdf_parcelles[C_ID + ['geometry']].copy()
for cl in classes_list:
    df_travail[cl] = 0.0

# 2. DÉFINITION DU MAPPING (Couches structurelles uniquement)
mapping_layers = {
    'BATIMENT': 'ARTIFICIALISE', 
    'CIMETIERE': 'ARTIFICIALISE', 
    'CONSTRUCTION_SURFACIQUE': 'ARTIFICIALISE', 
    'TERRAIN_DE_SPORT': 'ARTIFICIALISE',
    'SURFACE_HYDROGRAPHIQUE': 'EAU', 
    'RESERVOIR': 'ARTIFICIALISE',
    'ZONE_D_ESTRAN': 'NATURE'
}

# 3. TRAITEMENT DES COUCHES (INTERSECTIONS)
for layer_name, cat in mapping_layers.items():
    try:
        print(f"    > Intersection spatiale : {layer_name}")
        lyr_topo = gpd.read_file(TOPO_PATH, layer=layer_name, mask=gdf_parcelles, engine='pyogrio')
        
        if not lyr_topo.empty:
            if lyr_topo.crs != df_travail.crs:
                lyr_topo = lyr_topo.to_crs(df_travail.crs)
            
            inter = gpd.overlay(df_travail, lyr_topo, how='intersection')
            if not inter.empty:
                inter['S_INT'] = inter.geometry.area
                sums = inter.groupby(C_ID)['S_INT'].sum()
                
                # Agrégation
                df_travail.set_index(C_ID, inplace=True)
                df_travail[cat] = df_travail[cat].add(sums, fill_value=0)
                df_travail.reset_index(inplace=True)
    except Exception as e:
        print(f"      ! Information : Couche {layer_name} non traitée : {e}")

# 4. CONSOLIDATION ET GÉNÉRATION DES INDICATEURS
print("    > Consolidation des indicateurs statistiques...")

# Calcul surface totale
df_travail['SURF_TOTALE_M2'] = df_travail.geometry.area

# Calcul de 'AUTRE' (Tout ce qui n'est pas Bâtiment, Eau ou Estran tombera ici)
surfaces_identifiees = df_travail[["ARTIFICIALISE", "AGRICULTURE", "NATURE", "BOIS", "EAU"]].sum(axis=1)
df_travail['AUTRE'] = (df_travail['SURF_TOTALE_M2'] - surfaces_identifiees).clip(lower=0)

# Calcul des pourcentages
for cl in classes_list:
    df_travail[f"{cl}_%"] = ((df_travail[cl] / df_travail['SURF_TOTALE_M2']) * 100).round(2)

# Détermination dominante et probabilité
df_travail['DOMINANTE'] = df_travail[classes_list].idxmax(axis=1)
df_travail['PROBA_DOMINANTE'] = np.where(df_travail['DOMINANTE'] == 'AUTRE', 0, 100)

# Identifiant PL
df_travail['PL'] = (
    df_travail['CODE_DEP'].astype(str).str.zfill(2) + 
    df_travail['CODE_COM'].astype(str).str.zfill(3) + 
    df_travail['SECTION'].astype(str).str.zfill(2) + 
    df_travail['NUMERO'].astype(str).str.zfill(4)
)

# Export
output_path = Path(OUT_DIR) / "STAT_PARCELLES_BDTOPO_2025.csv"
cols_export = ['PL'] + C_ID + [f"{cl}_%" for cl in classes_list] + ["DOMINANTE", "PROBA_DOMINANTE", "SURF_TOTALE_M2"]
df_travail[cols_export].to_csv(output_path, sep=';', index=False, encoding='utf-8')

print(f">>> ANALYSE TERMINÉE. FICHIER GÉNÉRÉ : {output_path.name}")

>>> DÉMARRAGE DE L'ANALYSE BD TOPO (SANS VÉGÉTATION)
    > Intersection spatiale : BATIMENT


C:\Users\liege\anaconda3\Lib\site-packages\geopandas\io\file.py:576: UserWarning: Error parsing datetimes, original strings are returned: Out of bounds nanosecond timestamp: 1500-01-01, at position 92. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.
  return pyogrio.read_dataframe(path_or_bytes, bbox=bbox, **kwargs)
C:\Users\liege\anaconda3\Lib\site-packages\geopandas\tools\overlay.py:358: UserWarning: `keep_geom_type=True` in overlay resulted in 7 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  result = _collection_extract(result, geom_type, keep_geom_type_warning)


    > Intersection spatiale : CIMETIERE
    > Intersection spatiale : CONSTRUCTION_SURFACIQUE
    > Intersection spatiale : TERRAIN_DE_SPORT
    > Intersection spatiale : SURFACE_HYDROGRAPHIQUE
    > Intersection spatiale : RESERVOIR
    > Intersection spatiale : ZONE_D_ESTRAN
    > Consolidation des indicateurs statistiques...
>>> ANALYSE TERMINÉE. FICHIER GÉNÉRÉ : STAT_PARCELLES_BDTOPO_2025.csv


# VI. Consolidation et synthèse temporelle du registre parcellaire (RPG)

### 6.1. Module 1 : Fusion multi-sources (Standard + Complet)
Ce module traite le RPG comme un assemblage de sources distinctes et complémentaires. À l'instar de la BD TOPO, le script interroge deux flux indépendants pour caractériser une même unité foncière :
* **Indépendance des flux** : Le **RPG Standard** et le **RPG Complet** sont chargés et intersectés successivement. Le Standard assure la structure administrative globale, tandis que le Complet apporte des précisions sur ce qui n'apparaît pas dans le premier flux.
* **Agrégation spatiale** : Les surfaces issues de chaque source sont cumulées par parcelle. L'utilisation de l'identifiant `PL` garantit que chaque apport est correctement localisé sans créer de doublons géométriques, chaque source venant enrichir le bilan surfacique de la parcelle.
* **Fiabilité différenciée** : 
   $$PROBA\_PONDEREE = \frac{\sum (Surface \cap \times Confiance)}{\sum Surface \cap}$$
   Une confiance de 100 est assignée au Standard. Pour le Complet, la probabilité native est conservée. Le reliquat non couvert par ces deux sources est classé en **"AUTRE"** (confiance 0).



### 6.2. Module 2 : Arbitrage temporel et stabilité (2018-2024)
L'objectif est de définir l'usage agricole stable sur le long terme à partir des données consolidées annuellement.
* **Logique d'arbitrage** : 
    1. **Fréquence** : La classe la plus présente sur la période est élue (majorité).
    2. **Récence** : En cas d'égalité, le script utilise la chronologie inverse (depuis 2024) pour trancher.
* **Indicateurs** : Calcul du **Score de stabilité** et de la **Proba moyenne** pour valider la robustesse du diagnostic.

### 6.3. Justification méthodologique
Cette approche traite le RPG comme un système multi-sources cohérent. La distinction entre Standard et Complet permet d'exploiter chaque gisement de données sans perte d'information. La neutralité de la classe "AUTRE" assure qu'en l'absence de déclaration agricole, le système laisse la priorité aux autres référentiels lors de la fusion finale.

In [6]:
# =============================================================================
# MODULE : FUSION HARMONISÉE RPG
# =============================================================================
import pandas as pd
import geopandas as gpd
import numpy as np
from pathlib import Path

print("--- DÉMARRAGE DE LA FUSION HARMONISÉE RPG ---")

# 1. PARAMÉTRAGE ET RÉFÉRENTIELS
C_ID = config['CHAMPS_ID']
ANNEES_RPG = config['ANNEES_RPG']
PATH_RPG_STD = config['RPG_GPKG']
PATH_RPG_COMP = config['RPG_COMP_GPKG']
DIR_FUSION = config['DIR_CSV_RPG_FUS']

# Nomenclature (On garde BOIS dans la liste pour éviter les erreurs si appelé ailleurs)
CLASSES_CIBLES = ["ARTIFICIALISE", "AGRICULTURE", "NATURE", "BOIS", "EAU", "AUTRE"]

def reclass_unifie(row, est_std):
    """
    Standardisation selon tes règles spécifiques avec protection technique.
    """
    if est_std:
        # --- CAS 1 : RPG STANDARD (CODE_GROUP) ---
        try:
            val = row.get("CODE_GROUP", 0)
            grp = int(val) if val is not None else 0
        except (ValueError, TypeError):
            grp = 0
            
        if grp == 17: return "NATURE"
        return "AGRICULTURE"
        
    else:
        # --- CAS 2 : RPG COMPLET (culture) ---
        label = str(row.get("culture", ""))
        if not label or label == "None":
            return "AUTRE"
        
        l_low = label.lower()
        
        # --- RÈGLES STRICTES ---
        if "jardin collectif" in l_low: return "ARTIFICIALISE"
        if "landes" in l_low: return "NATURE"
        if "landes (pelouses)" in l_low: return "NATURE"    
        
        return "AGRICULTURE"

# --- EXÉCUTION DU FLUX TEMPOREL ---
for annee in ANNEES_RPG:
    print(f"\n>>> Traitement de l'annuité : {annee}")
    
    # PROTECTION : Nettoyage des identifiants (espaces, types)
    df_bilan = gdf_parcelles[C_ID + ['geometry']].copy()
    for col in C_ID:
        df_bilan[col] = df_bilan[col].astype(str).str.strip()
        
    df_bilan['SURF_CAD'] = df_bilan.geometry.area
    
    for cl in CLASSES_CIBLES:
        df_bilan[f"{cl}_s"] = 0.0
        df_bilan[f"{cl}_p"] = 0.0

    def traiter_source(path, layer_name, est_std):
        global df_bilan
        try:
            lyr_src = gpd.read_file(path, layer=layer_name, mask=gdf_parcelles, engine='pyogrio')
            
            if not lyr_src.empty:
                if lyr_src.crs != df_bilan.crs:
                    lyr_src = lyr_src.to_crs(df_bilan.crs)
                
                lyr_src['geometry'] = lyr_src.geometry.make_valid()
                inter = gpd.overlay(lyr_src, df_bilan[C_ID + ['geometry']], how='intersection')
                
                if not inter.empty:
                    # PROTECTION : Nettoyage post-intersection
                    for col in C_ID:
                        inter[col] = inter[col].astype(str).str.strip()
                        
                    inter['S_INT'] = inter.geometry.area
                    inter['CLASSE_UNIF'] = inter.apply(lambda r: reclass_unifie(r, est_std), axis=1)
                    
                    # PROTECTION : Correction technique des probabilités (np.where)
                    if not est_std and 'proba' in inter.columns:
                        p_raw = inter['proba'].astype(str).str.replace(',', '.').astype(float).fillna(0.0)
                        inter['p_val'] = np.where(p_raw <= 1.0, p_raw * 100.0, p_raw)
                    else:
                        inter['p_val'] = 100.0
                    
                    inter['P_POND'] = inter['S_INT'] * inter['p_val']
                    
                    for cl in CLASSES_CIBLES:
                        mask_cl = inter['CLASSE_UNIF'] == cl
                        if mask_cl.any():
                            subset = inter[mask_cl].groupby(C_ID).agg({'S_INT': 'sum', 'P_POND': 'sum'})
                            df_bilan = df_bilan.set_index(C_ID)
                            df_bilan[f"{cl}_s"] = df_bilan[f"{cl}_s"].add(subset['S_INT'], fill_value=0)
                            df_bilan[f"{cl}_p"] = df_bilan[f"{cl}_p"].add(subset['P_POND'], fill_value=0)
                            df_bilan = df_bilan.reset_index()
                            
        except Exception as e:
            print(f"   ! Note : La couche {layer_name} n'a pas pu être traitée (Ignoré).")

    traiter_source(PATH_RPG_STD, f"RPG_{annee}", est_std=True)
    traiter_source(PATH_RPG_COMP, f"RPG_COMPLETE_{annee}", est_std=False)

    # Finalisation
    surf_rpg_tot = df_bilan[[f"{cl}_s" for cl in CLASSES_CIBLES]].sum(axis=1)
    df_bilan['AUTRE_s'] = (df_bilan['SURF_CAD'] - surf_rpg_tot).clip(lower=0)
    
    for cl in CLASSES_CIBLES:
        df_bilan[f"{cl}_%"] = ((df_bilan[f"{cl}_s"] / df_bilan['SURF_CAD']) * 100).round(2)
    
    # PROTECTION : Dominante sécurisée
    df_bilan['DOMINANTE'] = np.where(
        surf_rpg_tot > 0,
        df_bilan[[f"{cl}_s" for cl in CLASSES_CIBLES]].idxmax(axis=1).str.replace('_s', ''),
        "AUTRE"
    )
    
    def calculer_proba(row):
        dom = row['DOMINANTE']
        if dom == "AUTRE": return 0.0
        return round(row[f"{dom}_p"] / row[f"{dom}_s"], 1) if row[f"{dom}_s"] > 0 else 0.0

    df_bilan['PROBA_PONDEREE'] = df_bilan.apply(calculer_proba, axis=1)

    # Identifiant PL propre
    df_bilan['PL'] = (
        df_bilan['CODE_DEP'].astype(str).str.zfill(2) + 
        df_bilan['CODE_COM'].astype(str).str.zfill(3) + 
        df_bilan['SECTION'].astype(str).str.zfill(2) + 
        df_bilan['NUMERO'].astype(str).str.zfill(4)
    )

    path_out = Path(DIR_FUSION) / f"STAT_RPG_SYNTHESE_{annee}.csv"
    cols = ['PL'] + C_ID + [f"{cl}_%" for cl in CLASSES_CIBLES] + ["DOMINANTE", "PROBA_PONDEREE", "SURF_CAD"]
    
    path_out.parent.mkdir(parents=True, exist_ok=True)
    df_bilan[cols].to_csv(path_out, sep=';', index=False, encoding='utf-8')
    print(f"   -> Fichier généré : {path_out.name}")

print("\n--- ANALYSE TERMINÉE : SÉCURITÉ ACTIVE & RÈGLES APPLIQUÉES ---")

--- DÉMARRAGE DE LA FUSION HARMONISÉE RPG ---

>>> Traitement de l'annuité : 2018
   -> Fichier généré : STAT_RPG_SYNTHESE_2018.csv

>>> Traitement de l'annuité : 2019
   -> Fichier généré : STAT_RPG_SYNTHESE_2019.csv

>>> Traitement de l'annuité : 2020
   -> Fichier généré : STAT_RPG_SYNTHESE_2020.csv

>>> Traitement de l'annuité : 2021
   -> Fichier généré : STAT_RPG_SYNTHESE_2021.csv

>>> Traitement de l'annuité : 2022
   -> Fichier généré : STAT_RPG_SYNTHESE_2022.csv

>>> Traitement de l'annuité : 2023
   ! Note : La couche RPG_COMPLETE_2023 n'a pas pu être traitée (Ignoré).
   -> Fichier généré : STAT_RPG_SYNTHESE_2023.csv

>>> Traitement de l'annuité : 2024
   ! Note : La couche RPG_COMPLETE_2024 n'a pas pu être traitée (Ignoré).
   -> Fichier généré : STAT_RPG_SYNTHESE_2024.csv

--- ANALYSE TERMINÉE : SÉCURITÉ ACTIVE & RÈGLES APPLIQUÉES ---


In [7]:
# =============================================================================
# MODULE : SYNTHÈSE TEMPORELLE RPG (ARBITRAGE ET STABILITÉ)
# =============================================================================
import pandas as pd
from collections import defaultdict, Counter
from pathlib import Path

print(">>> DÉMARRAGE DE LA SYNTHÈSE TEMPORELLE RPG")

# --- 1. SÉCURISATION DE LA CONFIGURATION ---
DIR_FUSION = config.get('DIR_CSV_RPG_FUS')
DIR_SORTIE = config.get('DIR_SORTIE', globals().get('DIR_SORTIE'))
CHAMPS_ID  = config.get('CHAMPS_ID')
ANNEES_RPG = sorted([int(a) for a in config.get('ANNEES_RPG', [])])

if not DIR_SORTIE:
    raise ValueError("Erreur : DIR_SORTIE n'est pas défini dans la config.")

fichier_synthese = Path(DIR_SORTIE) / "SYNTHESE_RPG_FREQUENCE.csv"
base_globale = defaultdict(dict)

# --- 2. LECTURE DES DONNÉES ANNUELLES ---
print(f"    > Lecture des historiques ({min(ANNEES_RPG)} - {max(ANNEES_RPG)})...")
for annee in ANNEES_RPG:
    chemin = Path(DIR_FUSION) / f"STAT_RPG_SYNTHESE_{annee}.csv"
    if not chemin.exists(): continue
    
    # Lecture forcée en string pour préserver les zéros (01, 005...)
    df_annuel = pd.read_csv(chemin, sep=';', encoding='utf-8', dtype={c: str for c in CHAMPS_ID})
    
    for _, row in df_annuel.iterrows():
        # --- RÉPARATION DES IDENTIFIANTS (Sécurité Dalles 1 & 4) ---
        # On crée un dictionnaire temporaire pour formater chaque champ proprement
        d = {c: str(row[c]).strip() for c in CHAMPS_ID}
        
        # Application du zfill sur les codes numériques pour l'alignement Cadastre
        if 'CODE_DEP' in d: d['CODE_DEP'] = d['CODE_DEP'].zfill(2)
        if 'CODE_COM' in d: d['CODE_COM'] = d['CODE_COM'].zfill(3)
        if 'SECTION' in d:  d['SECTION']  = d['SECTION'].zfill(2)
        if 'NUMERO' in d:   d['NUMERO']   = d['NUMERO'].zfill(4)
        
        # La clé est un tuple des valeurs dans l'ordre de CHAMPS_ID
        cle = tuple(d[c] for c in CHAMPS_ID)
        
        try:
            proba = float(str(row['PROBA_PONDEREE']).replace(',', '.'))
        except:
            proba = 0.0
            
        base_globale[cle][annee] = {
            'DOM': row.get('DOMINANTE', 'AUTRE'),
            'PROB': proba
        }

# --- 3. ARBITRAGE ET CALCULS ---
print("    > Calcul de l'arbitrage (Majorité vs Récence)...")
final_data = []

for cle, data in base_globale.items():
    historique = {a: data[a]['DOM'] for a in ANNEES_RPG if a in data}
    if not historique: continue

    classes_obs = list(historique.values())
    counts = Counter(classes_obs)
    
    max_freq = max(counts.values())
    gagnants = [cl for cl, nb in counts.items() if nb == max_freq]

    if len(gagnants) == 1:
        elu = gagnants[0]
        methode = "MAJORITE"
    else:
        elu = next((historique[a] for a in reversed(ANNEES_RPG) if a in historique and historique[a] in gagnants), None)
        methode = "RECENCE_EGALITE"

    score_stabilite = round((counts[elu] / len(ANNEES_RPG)) * 100, 1)
    probas_elu = [data[a]['PROB'] for a in data if a in historique and historique[a] == elu]
    proba_moyenne = round(sum(probas_elu) / len(probas_elu), 1) if probas_elu else 0.0

    # --- RECONSTRUCTION DU PL (Basée sur tes colonnes précises) ---
    # On crée un mapping pour retrouver facilement les valeurs par nom de colonne
    m = dict(zip(CHAMPS_ID, cle))
    
    # PL = DEP(2) + COM(3) + SECTION(2) + NUMERO(4)
    # On utilise .get() pour éviter tout plantage si une colonne manquait
    id_pl = m.get('CODE_DEP', '') + m.get('CODE_COM', '') + m.get('SECTION', '') + m.get('NUMERO', '')
    
    ligne = {'PL': id_pl}
    # Remplissage dynamique pour éviter l'IndexError
    for i, champ in enumerate(CHAMPS_ID):
        ligne[champ] = cle[i]
        
    for a in ANNEES_RPG:
        ligne[f"DOM_{a}"] = historique.get(a, "N/A")
        ligne[f"PROB_{a}"] = round(data[a]['PROB'], 1) if a in data else 0.0
            
    ligne.update({
        'DOMINANTE': elu,
        'SCORE_STABILITE': score_stabilite,
        'PROBA_PONDEREE': proba_moyenne,
        'LOGIQUE_ARBITRAGE': methode
    })
    
    final_data.append(ligne)

# --- 4. EXPORT ---
df_final = pd.DataFrame(final_data)
df_final.to_csv(fichier_synthese, sep=';', index=False, encoding='utf-8')

print(f">>> SYNTHÈSE TERMINÉE : {fichier_synthese.name}")

>>> DÉMARRAGE DE LA SYNTHÈSE TEMPORELLE RPG
    > Lecture des historiques (2018 - 2024)...
    > Calcul de l'arbitrage (Majorité vs Récence)...
>>> SYNTHÈSE TERMINÉE : SYNTHESE_RPG_FREQUENCE.csv


# VII. Analyse spatiale et synthèse temporelle COSIA

### 7.1. Module 1 : Analyse statistique annuelle (Structure miroir)
Ce module traite les données COSIA selon une architecture calquée sur celle de l'OSO, permettant une comparaison directe des résultats. Le processus se distingue par :
* **Mapping sémantique rigoureux** : Conversion des codes sources COSIA vers la nomenclature cible (6 classes) avec un nettoyage systématique des caractères (suppression des points) pour garantir la correspondance.
* **Intégration de probabilités expertes** : Contrairement à l'OSO , le module COSIA utilise des **probabilités fixes** définies dans la nomenclature. Chaque usage du sol reçoit un indice de fiabilité théorique (`PROBA_VAL`) reflétant la précision historique de la donnée source.
* **Calcul de fiabilité pondérée** : 
  $$PROBA\_DOMINANTE = \frac{\sum (Surface \cap \times Confiance\_fixe)}{\sum Surface \cap}$$

### 7.2. Module 2 : Synthèse temporelle et arbitrage de cohérence
L'objectif est de dégager une tendance stable sur l'ensemble des millésimes COSIA disponibles pour consolider le diagnostic foncier.
* **Logique de Décision (Majorité vs Récence)** : 
    1. **Fréquence** : Élection de la classe la plus représentée sur l'historique pour lisser les variations mineures.
    2. **Récence si égalité** : En cas d'ex-æquo (ex: 2 ans Nature / 2 ans Agriculture), le script applique une chronologie inverse en interrogeant le millésime le plus récent pour refléter l'état actuel.
* **Indicateurs de Performance** :
    * **SCORE_STABILITE** : Mesure du taux de permanence de la classe élue au fil des années.
    * **PROBA_MOYENNE** : Synthèse des indices de fiabilité de la nomenclature pour la classe retenue.

### 7.3. Justification méthodologique
L'analyse COSIA agit comme un **référentiel de validation**. En reproduisant la structure de l'OSO, elle permet de croiser deux visions de l'occupation du sol. La standardisation de l'identifiant `PL` avec formatage `zfill` assure que ces données sont prêtes pour l'arbitrage final avec le RPG et la BD TOPO etc...

In [4]:
# =============================================================================
# MODULE : ANALYSE STATISTIQUE COSIA (STRUCTURE MIROIR OSO)
# =============================================================================
import pandas as pd
import geopandas as gpd

# --- 1. PREPARATION DES REFERENTIELS ET NOMENCLATURES ---
classes_list = ["ARTIFICIALISE", "AGRICULTURE", "NATURE", "BOIS", "EAU", "AUTRE"]

# Chargement nomenclature COSIA
df_nomen = pd.read_csv(config['NOMEN_COSIA_CSV'], sep=';', encoding='utf-8-sig')

# Mapping des Classes et des Probabilités fixes issues de la nomenclature
# On nettoie le code source (suppression des points) pour la correspondance
mapping_cosia = {str(row['CODE_SOURCE']).replace('.', '').strip(): str(row['NATURE_SOL']).strip() 
                 for _, row in df_nomen.iterrows()}

mapping_proba = {str(row['CODE_SOURCE']).replace('.', '').strip(): float(str(row['PROBA_SOURCE']).replace(',', '.')) 
                 for _, row in df_nomen.iterrows()}

# --- 2. TRAITEMENT ANALYTIQUE ANNUEL ---
for annee in config['ANNEES_COSIA']:
    print(f"INFO : Analyse spatiale COSIA - Millesime {annee}")
    layer_name = f"COSIA_VECTEUR_{annee}"
    
    try:
        # Lecture avec mask pour le minimum d'optimisation vital
        lyr_cosia = gpd.read_file(config['COSIA_GPKG'], layer=layer_name, mask=gdf_parcelles, engine='pyogrio')
        if lyr_cosia.crs != gdf_parcelles.crs:
            lyr_cosia = lyr_cosia.to_crs(gdf_parcelles.crs)
    except:
        print(f"ERREUR : Couche {layer_name} introuvable.")
        continue

    # Operation d'intersection geometrique
    inter = gpd.overlay(lyr_cosia, gdf_parcelles, how='intersection')
    if inter.empty:
        continue
    
    # Calcul des surfaces et mapping
    inter['S_INT'] = inter.geometry.area
    
    # Nettoyage du champ 'numero' spécifique à COSIA pour le mapping
    inter['num_clean'] = inter['numero'].astype(str).str.replace('.', '', regex=False).str.strip()
    
    inter['CLASSE_NOM'] = inter['num_clean'].map(mapping_cosia).fillna("AUTRE")
    inter['PROBA_VAL'] = inter['num_clean'].map(mapping_proba).fillna(0.0)
    
    # Préparation du cumul pour la probabilité moyenne
    inter['P_POND'] = inter['S_INT'] * inter['PROBA_VAL']

    # Agregation statistique par unite fonciere
    stats = inter.groupby(config['CHAMPS_ID'] + ['CLASSE_NOM']).agg({
        'S_INT': 'sum',
        'P_POND': 'sum'
    }).reset_index()

    # Pivotage des donnees
    df_res = stats.pivot_table(index=config['CHAMPS_ID'], columns='CLASSE_NOM', values='S_INT', aggfunc='sum').fillna(0)
    
    for cl in classes_list:
        if cl not in df_res.columns: df_res[cl] = 0.0

    # Calcul des indicateurs de synthese (Dénominateur = Surface détectée)
    df_res['SURF_TOTALE_M2'] = df_res[classes_list].sum(axis=1).round(3)
    
    # Sécurité pour éviter la division par zéro sur les parcelles vides
    for cl in classes_list:
        df_res[f"{cl}_%"] = 0.0
        df_res.loc[df_res['SURF_TOTALE_M2'] > 0, f"{cl}_%"] = \
            ((df_res[cl] / df_res['SURF_TOTALE_M2']) * 100).round(3)

    # Détermination de la dominante (Sécurisée contre les lignes à 0)
    df_res['DOMINANTE'] = df_res[classes_list].idxmax(axis=1)
    df_res.loc[df_res['SURF_TOTALE_M2'] == 0, 'DOMINANTE'] = "AUTRE"

    # Calcul de la probabilité moyenne sur la classe dominante
    proba_pivot = stats.pivot_table(index=config['CHAMPS_ID'], columns='CLASSE_NOM', values='P_POND', aggfunc='sum').fillna(0)
    
    def get_proba(row):
        dom = row['DOMINANTE']
        s_dom = row[dom]
        try:
            cumul = proba_pivot.loc[row.name, dom] if row.name in proba_pivot.index else 0
            return round(cumul / s_dom, 3) if s_dom > 0 else 0
        except:
            return 0

    df_res['PROBA_DOMINANTE'] = df_res.apply(get_proba, axis=1)

    # --- 3. GENERATION DU CHAMP 'PL' (CONCATENATION SIMPLE) ---
    df_res = df_res.reset_index()
    df_res['PL'] = (
        df_res['CODE_DEP'].astype(str).str.strip() + 
        df_res['CODE_COM'].astype(str).str.strip() + 
        df_res['SECTION'].astype(str).str.strip() + 
        df_res['NUMERO'].astype(str).str.strip()
    )

    # --- 4. EXPORTATION DES RESULTATS ---
    output_csv = config['DOSSIER_CSV_COSIA'] / f"STAT_PARCELLES_COSIA_{annee}.csv"
    cols_finales = ['PL'] + config['CHAMPS_ID'] + [f"{cl}_%" for cl in classes_list] + \
                   ["DOMINANTE", "PROBA_DOMINANTE", "SURF_TOTALE_M2"]
    
    df_res[cols_finales].to_csv(output_csv, sep=';', index=False, encoding='utf-8')
    print(f"SUCCES : Export COSIA millesime {annee} genere.")

print("TRAITEMENT COSIA TERMINE.")

INFO : Analyse spatiale COSIA - Millesime 2018


C:\Users\liege\anaconda3\Lib\site-packages\geopandas\tools\overlay.py:358: UserWarning: `keep_geom_type=True` in overlay resulted in 45 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  result = _collection_extract(result, geom_type, keep_geom_type_warning)


SUCCES : Export COSIA millesime 2018 genere.
INFO : Analyse spatiale COSIA - Millesime 2021


C:\Users\liege\anaconda3\Lib\site-packages\geopandas\tools\overlay.py:358: UserWarning: `keep_geom_type=True` in overlay resulted in 3 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  result = _collection_extract(result, geom_type, keep_geom_type_warning)


SUCCES : Export COSIA millesime 2021 genere.
INFO : Analyse spatiale COSIA - Millesime 2024


C:\Users\liege\anaconda3\Lib\site-packages\geopandas\tools\overlay.py:358: UserWarning: `keep_geom_type=True` in overlay resulted in 6 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  result = _collection_extract(result, geom_type, keep_geom_type_warning)


SUCCES : Export COSIA millesime 2024 genere.
TRAITEMENT COSIA TERMINE.


In [5]:
# =============================================================================
# MODULE : SYNTHÈSE TEMPORELLE COSIA
# Description : Consolidation des millésimes COSIA et calcul du score de stabilité.
# =============================================================================
import pandas as pd
from collections import defaultdict
from pathlib import Path

print(">>> DÉMARRAGE DE LA SYNTHÈSE TEMPORELLE COSIA")

# 1. SÉCURISATION DE LA CONFIGURATION
# Récupération des chemins avec gestion des clés manquantes
DIR_COSIA = config.get('DOSSIER_CSV_COSIA')
# Sécurité : on cherche la clé dans config, sinon on utilise la variable globale
DIR_SORTIE = config.get('DIR_SORTIE', globals().get('DIR_SORTIE'))
CHAMPS_ID  = config.get('CHAMPS_ID')
ANNEES_COSIA = sorted([int(a) for a in config.get('ANNEES_COSIA', [])])

if not DIR_SORTIE:
    raise ValueError("Erreur : DIR_SORTIE n'est pas défini dans la configuration.")

fichier_synthese = Path(DIR_SORTIE) / "SYNTHESE_COSIA_FREQUENCE.csv"
base_globale = defaultdict(dict)

# 2. CONSOLIDATION DES DONNÉES ANNUELLES
print(f"    > Lecture des millésimes : {ANNEES_COSIA}...")
for annee in ANNEES_COSIA:
    chemin = Path(DIR_COSIA) / f"STAT_PARCELLES_COSIA_{annee}.csv"
    if not chemin.exists():
        print(f"    ! Millésime {annee} non trouvé, passage à l'année suivante.")
        continue
    
    df = pd.read_csv(chemin, sep=';', encoding='utf-8')
    for _, row in df.iterrows():
        # Clé unique basée sur les identifiants pivots
        cle = tuple(str(row[c]).strip() for c in CHAMPS_ID)
        base_globale[cle][annee] = {
            'DOM': row.get('DOMINANTE', 'AUTRE'),
            'PROB': float(row.get('PROBA_DOMINANTE', 0.0))
        }

# 3. ARBITRAGE STATISTIQUE ET EXPORTATION
print("    > Calcul de la décision finale et des indices de confiance...")
final_rows = []

for cle, data in base_globale.items():
    historique = [data[a]['DOM'] for a in ANNEES_COSIA if a in data]
    if not historique: continue

    # Calcul des fréquences d'apparition
    counts = defaultdict(int)
    for c in historique: counts[c] += 1
    max_freq = max(counts.values())
    gagnants = [cl for cl, nb in counts.items() if nb == max_freq]

    # Application des règles d'arbitrage (Majorité vs Récence)
    if len(gagnants) == 1:
        elu, methode = gagnants[0], "MAJORITE"
    else:
        # Sélection de la classe la plus récente parmi les gagnants
        elu = next((data[a]['DOM'] for a in reversed(ANNEES_COSIA) 
                    if a in data and data[a]['DOM'] in gagnants), "AUTRE")
        methode = "RECENT_SI_EGALITE"

    # Calcul des indicateurs de fiabilité
    # SCORE_STABILITE : pourcentage de permanence de la classe élue
    score_stabilite = round((counts[elu] / len(ANNEES_COSIA)) * 100, 1)
    
    # PROBA_MOYENNE : moyenne des probabilités sources pour la classe élue
    probas_elu = [data[a]['PROB'] for a in data if a in data and data[a]['DOM'] == elu]
    proba_moyenne = round(sum(probas_elu) / len(probas_elu), 3) if probas_elu else 0.0

    # Reconstruction de l'identifiant standardisé PL
    # Formatage : Dep(2) + Com(3) + Sec(2) + Num(4)
    id_pl = "".join([cle[0].zfill(2), cle[2].zfill(3), cle[3].zfill(2), cle[4].zfill(4)])
    
    # Structuration de la ligne de sortie
    res = {'PL': id_pl}
    res.update({CHAMPS_ID[i]: cle[i] for i in range(len(CHAMPS_ID))})
    
    for a in ANNEES_COSIA:
        res[f"DOM_{a}"] = data.get(a, {}).get('DOM', "N/A")
        res[f"PROB_{a}"] = data.get(a, {}).get('PROB', 0.0)
    
    res.update({
        'COSIA_CLASSE_FINALE': elu, 
        'SCORE_STABILITE': score_stabilite, 
        'PROBA_MOYENNE': proba_moyenne, 
        'METHODE_ARBITRAGE': methode
    })
    final_rows.append(res)

# 4. EXPORTATION FINALE
df_final = pd.DataFrame(final_rows)
df_final.to_csv(fichier_synthese, sep=';', index=False, encoding='utf-8')

print(f"\n>>> SYNTHÈSE TERMINÉE. FICHIER GÉNÉRÉ : {fichier_synthese.name}")

>>> DÉMARRAGE DE LA SYNTHÈSE TEMPORELLE COSIA
    > Lecture des millésimes : [2018, 2021, 2024]...
    > Calcul de la décision finale et des indices de confiance...

>>> SYNTHÈSE TERMINÉE. FICHIER GÉNÉRÉ : SYNTHESE_COSIA_FREQUENCE.csv
